# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hafar al-batin,28.4328,45.9708,7.32,70,0,1.03,SA,1736901583
1,1,olonkinbyen,70.9221,-8.7187,-0.37,99,100,5.21,SJ,1736901661
2,2,berdigestyakh,62.0989,126.6992,-30.52,88,100,4.79,RU,1736901661
3,3,margaret river,-33.9500,115.0667,18.25,53,55,1.39,AU,1736901588
4,4,saint-philippe,-21.3585,55.7679,25.85,81,95,5.80,RE,1736901661


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    color = 'City',
    alpha = 0.5
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[(city_data_df['Max Temp'] < 30) &
                        (city_data_df['Max Temp'] > 23) & 
                        (city_data_df['Humidity'] < 80) & 
                        (city_data_df['Humidity'] > 40) &
                        (city_data_df['Wind Speed'] < 5) &
                        (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,wailua homesteads,22.0669,-159.3780,26.44,75,0,4.63,US,1736901662
166,166,lihue,21.9789,-159.3672,26.77,69,0,4.63,US,1736901685
170,170,piton saint-leu,-21.2196,55.3151,24.06,73,0,2.57,RE,1736901686
236,236,kapa'a,22.0752,-159.3190,27.18,75,0,4.63,US,1736901695
258,258,urangan,-25.3000,152.9000,25.88,77,0,1.99,AU,1736901700
373,373,puerto san jose,13.9256,-90.8244,29.03,77,0,2.06,GT,1736901724
398,398,coxim,-18.5067,-54.7600,25.87,74,0,0.69,BR,1736901727
417,417,cordoba,-31.4135,-64.1811,25.47,61,0,3.09,AR,1736901458
438,438,saint-pierre,-21.3393,55.4781,25.82,73,0,2.57,RE,1736901733


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
11,wailua homesteads,US,22.0669,-159.3780,75,
166,lihue,US,21.9789,-159.3672,69,
170,piton saint-leu,RE,-21.2196,55.3151,73,
236,kapa'a,US,22.0752,-159.3190,75,
258,urangan,AU,-25.3000,152.9000,77,
373,puerto san jose,GT,13.9256,-90.8244,77,
398,coxim,BR,-18.5067,-54.7600,74,
417,cordoba,AR,-31.4135,-64.1811,61,
438,saint-pierre,RE,-21.3393,55.4781,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
params = {'categories':categories,
          'apiKey':geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary and convert response to JSON format
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
lihue - nearest hotel: Kauai Palms
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
kapa'a - nearest hotel: Pono Kai Resort
urangan - nearest hotel: Ramada
puerto san jose - nearest hotel: Martita
coxim - nearest hotel: Avendita
cordoba - nearest hotel: Hotel Garden
saint-pierre - nearest hotel: Tropic Hotel


,City,Country,Lat,Lng,Humidity,Hotel,Hotel Name
11,wailua homesteads,US,22.0669,-159.3780,75,,Hilton Garden Inn Kauai Wailua Bay
166,lihue,US,21.9789,-159.3672,69,,Kauai Palms
170,piton saint-leu,RE,-21.2196,55.3151,73,,Appartement Piton Saint Leu
236,kapa'a,US,22.0752,-159.3190,75,,Pono Kai Resort
258,urangan,AU,-25.3000,152.9000,77,,Ramada
373,puerto san jose,GT,13.9256,-90.8244,77,,Martita
398,coxim,BR,-18.5067,-54.7600,74,,Avendita
417,cordoba,AR,-31.4135,-64.1811,61,,Hotel Garden
438,saint-pierre,RE,-21.3393,55.4781,73,,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = 'City',
    size = 50,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)